# Output Calculated SWE NetCDFs

In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import xarray as xr
import pickle
import altair as alt
import os.path

### Load Data:

In [3]:
stations = pickle.load(open('../data/snv_dataframes/snv_temp_precip_cluster_gdf_snvs_quantiles.pkl','rb'))


/var/folders/s4/pymynk_14w75x2ghv0xjkqlm0000gn/T/ipykernel_65880/1155285028.py:1: DeprecationWarning: numpy.core.numeric is deprecated and has been renamed to numpy._core.numeric. The numpy._core namespace contains private NumPy internals and its use is discouraged, as NumPy internals can change without warning in any release. In practice, most real-world usage of numpy.core is to access functionality in the public NumPy API. If that is the case, use the public NumPy API. If not, you are using NumPy internals. If you would still like to access an internal attribute, use numpy._core.numeric._frombuffer.
  stations = pickle.load(open('../data/snv_dataframes/snv_temp_precip_cluster_gdf_snvs_quantiles.pkl','rb'))


In [2]:
swe = xr.open_dataset('../data/reanalysis/swe_reanalysis_ucrb.nc')
stations_ucrb = pickle.load(open('../data/snv_dataframes/snv_ucrb_clusters_gdf_snvs_quantiles.pkl','rb'))

ucrb = pickle.load(open('../data/misc/ucrb.pkl','rb'))

FileNotFoundError: [Errno 2] No such file or directory: '../data/snv_dataframes/snv_ucrb_clusters_gdf_snvs_quantiles.pkl'

## Calculate SWE using UCRB mean SNV and Quantile:

### SNV:

In [5]:
calculated_swe_mean_snv_ucrb_all = xr.full_like(swe.SWE_Post, np.nan)

In [6]:
mean_snv_ucrb_all = xr.open_dataset('../data/calculated_swe/intermediate_ncs/mean_snv_ucrb_all.nc')

In [13]:
for year in np.arange(0,32):
    to_exclude = [year]
    years = [element for i, element in enumerate(np.arange(1990,2022)) if i not in to_exclude]
    calculated_swe_mean_snv_ucrb_all[year] = mean_snv_ucrb_all.SWE_Post[year] * swe.SWE_Post.sel(Year=years).std(dim='Year', skipna=True, ddof=1).where(swe.SWE_Post.mean(dim='Year')>0.1) + swe.SWE_Post.sel(Year=years).mean(dim='Year').where(swe.SWE_Post.mean(dim='Year')>0.1)

In [14]:
calculated_swe_mean_snv_ucrb_all.to_netcdf('../data/calculated_swe/snv/calculated_swe_mean_snv_ucrb_all.nc')

### Quantile:

In [6]:
np.median(stations_ucrb[stations_ucrb.index.get_level_values(1)==(year + 1990)]['quantile_q8'].values)

0.26262626262626265

In [7]:
file_path = '../data/calculated_swe/quantiles/calculated_swe_quantile_median_all_q8.nc'

if os.path.isfile(file_path):
    calculated_swe_quantiles_xr_q8_median_all = xr.open_dataarray(file_path)
else:
    calculated_swe_quantiles_xr_q8_median_all = xr.full_like(swe.SWE_Post, np.nan)
    for year in np.arange(0,32):
        quantile_fn = f'../data/quantile_ncs/sr_quantiles_{year+1990}_q8.nc'
        sr_quantiles_yr = xr.open_dataarray(quantile_fn)
        calculated_swe_quantiles_xr_q8_median_all[year] = sr_quantiles_yr.interp(quantile=np.median(stations_ucrb[stations_ucrb.index.get_level_values(1)==(year + 1990)]['quantile_q8'].values), method='linear')
    calculated_swe_quantiles_xr_q8_median_all.to_netcdf(file_path) 

______
## Calculate SWE from the mean SNV and Quantile within each cluster for each year:

### Cluster definition: SWE Reanalysis WUS clusters at 8x coarsened:

### SNV:

In [3]:
mean_snv_wus_coarse = xr.open_dataset('../data/calculated_swe/intermediate_ncs/mean_snv_wus_coarse_wic_sc.nc')

In [4]:
calculated_swe_mean_snv_wus_coarse = xr.full_like(swe.SWE_Post, np.nan)

In [5]:
for year in np.arange(0,32):
    to_exclude = [year]
    years = [element for i, element in enumerate(np.arange(1990,2022)) if i not in to_exclude]
    calculated_swe_mean_snv_wus_coarse[year] = mean_snv_wus_coarse.SWE_Post[year] * swe.SWE_Post.sel(Year=years).std(dim='Year', skipna=True, ddof=1).where(swe.SWE_Post.mean(dim='Year')>0.1) + swe.SWE_Post.sel(Year=years).mean(dim='Year').where(swe.SWE_Post.mean(dim='Year')>0.1)

In [7]:
calculated_swe_mean_snv_wus_coarse.to_netcdf('../data/calculated_swe/snv/calculated_swe_mean_snv_wus_coarse2.nc')

### Quantile:

In [8]:
median_quantile_wus_coarse = xr.open_dataset('../data/calculated_swe/intermediate_ncs/median_quantile_wus_coarse_wic_sc.nc')

In [9]:
file_path = '../data/calculated_swe/quantiles/calculated_swe_quantile_median_wic_wus_coarse_q8_2.nc'

if os.path.isfile(file_path):
    calculated_swe_quantiles_wus_coarse_q8 = xr.open_dataarray(file_path)
else:
    calculated_swe_quantiles_wus_coarse_q8 = xr.full_like(swe.SWE_Post, np.nan)
    for year in np.arange(0,32):
        quantile_fn = f'../data/quantile_ncs/sr_quantiles_{year+1990}_q8.nc'
        sr_quantiles_yr = xr.open_dataarray(quantile_fn)
        calculated_swe_quantiles_wus_coarse_q8[year] = sr_quantiles_yr.interp(quantile=median_quantile_wus_coarse.SWE_Post[year], method='linear')
    calculated_swe_quantiles_wus_coarse_q8.to_netcdf(file_path)

### Cluster definition: SWE Reanalysis WUS clusters at original resolution:

### SNV:

In [26]:
mean_snv_wus_origres = xr.open_dataset('../data/calculated_swe/intermediate_ncs/mean_snv_wus_origres_wic_sc.nc')

In [27]:
calculated_swe_mean_snv_wus_origres = xr.full_like(swe.SWE_Post, np.nan)

In [28]:
for year in np.arange(0,32):
    to_exclude = [year]
    years = [element for i, element in enumerate(np.arange(1990,2022)) if i not in to_exclude]
    calculated_swe_mean_snv_wus_origres[year] = mean_snv_wus_origres.SWE_Post[year] * swe.SWE_Post.sel(Year=years).std(dim='Year', skipna=True, ddof=1).where(swe.SWE_Post.mean(dim='Year')>0.1) + swe.SWE_Post.sel(Year=years).mean(dim='Year').where(swe.SWE_Post.mean(dim='Year')>0.1)

In [29]:
calculated_swe_mean_snv_wus_origres.to_netcdf('../data/calculated_swe/snv/calculated_swe_mean_snv_wus_origres.nc')

### Quantile:

In [30]:
mean_quantile_wus_origres = xr.open_dataset('../data/calculated_swe/intermediate_ncs/mean_quantile_wus_origres_wic_sc.nc')

In [31]:
file_path = '../data/calculated_swe/quantiles/calculated_swe_quantile_mean_wic_wus_origres_q8.nc'

if os.path.isfile(file_path):
    calculated_swe_quantiles_wus_origres_q8 = xr.open_dataarray(file_path)
else:
    calculated_swe_quantiles_wus_origres_q8 = xr.full_like(swe.SWE_Post, np.nan)
    for year in np.arange(0,32):
        quantile_fn = f'../data/quantile_ncs/sr_quantiles_{year+1990}_q8.nc'
        sr_quantiles_yr = xr.open_dataarray(quantile_fn)
        calculated_swe_quantiles_wus_origres_q8[year] = sr_quantiles_yr.interp(quantile=mean_quantile_wus_origres.SWE_Post[year], method='linear')
    calculated_swe_quantiles_wus_origres_q8.to_netcdf(file_path)

### Cluster definition: SWE Reanalysis UCRB clusters at original resolution:

### SNV:

In [32]:
mean_snv_ucrb_origres = xr.open_dataset('../data/calculated_swe/intermediate_ncs/mean_snv_ucrb_origres_wic_sc.nc')

In [33]:
calculated_swe_mean_snv_ucrb_origres = xr.full_like(swe.SWE_Post, np.nan)

In [34]:
for year in np.arange(0,32):
    to_exclude = [year]
    years = [element for i, element in enumerate(np.arange(1990,2022)) if i not in to_exclude]
    calculated_swe_mean_snv_ucrb_origres[year] = mean_snv_ucrb_origres.SWE_Post[year] * swe.SWE_Post.sel(Year=years).std(dim='Year', skipna=True, ddof=1).where(swe.SWE_Post.mean(dim='Year')>0.1) + swe.SWE_Post.sel(Year=years).mean(dim='Year').where(swe.SWE_Post.mean(dim='Year')>0.1)

In [35]:
calculated_swe_mean_snv_ucrb_origres.to_netcdf('../data/calculated_swe/snv/calculated_swe_mean_snv_ucrb_origres.nc')

### Quantile:

In [36]:
mean_quantile_ucrb_origres = xr.open_dataset('../data/calculated_swe/intermediate_ncs/mean_quantile_ucrb_origres_wic_sc.nc')

In [37]:
file_path = '../data/calculated_swe/quantiles/calculated_swe_quantile_mean_wic_ucrb_origres_q8.nc'

if os.path.isfile(file_path):
    calculated_swe_quantiles_ucrb_origres_q8 = xr.open_dataarray(file_path)
else:
    calculated_swe_quantiles_ucrb_origres_q8 = xr.full_like(swe.SWE_Post, np.nan)
    for year in np.arange(0,32):
        quantile_fn = f'../data/quantile_ncs/sr_quantiles_{year+1990}_q8.nc'
        sr_quantiles_yr = xr.open_dataarray(quantile_fn)
        calculated_swe_quantiles_ucrb_origres_q8[year] = sr_quantiles_yr.interp(quantile=mean_quantile_ucrb_origres.SWE_Post[year], method='linear')
    calculated_swe_quantiles_ucrb_origres_q8.to_netcdf(file_path)

______
## Calculate SWE from the nearest station's SNV and Quantile for each year:

### SNV:

In [38]:
nearest_snv_nc = xr.open_dataset('../data/calculated_swe/intermediate_ncs/nearest_station_snv_nc.nc')

In [39]:
calculated_swe_nearest_snv_nc = xr.full_like(swe.SWE_Post, np.nan)

In [40]:
for year in np.arange(0,32):
    to_exclude = [year]
    years = [element for i, element in enumerate(np.arange(1990,2022)) if i not in to_exclude]
    calculated_swe_nearest_snv_nc[year] = nearest_snv_nc.SWE_Post[year] * swe.SWE_Post.sel(Year=years).std(dim='Year', skipna=True, ddof=1).where(swe.SWE_Post.mean(dim='Year')>0.1) + swe.SWE_Post.sel(Year=years).mean(dim='Year').where(swe.SWE_Post.mean(dim='Year')>0.1)

In [41]:
calculated_swe_nearest_snv_nc.to_netcdf('../data/calculated_swe/snv/calculated_swe_nearest_snv_nc.nc')

### Quantile:

In [42]:
nearest_station_quantile = xr.open_dataset('../data/calculated_swe/intermediate_ncs/nearest_station_quantile.nc')

In [43]:
file_path = '../data/calculated_swe/quantiles/calculated_swe_nearest_station_quantile_q8.nc'

if os.path.isfile(file_path):
    calculated_swe_nearest_station_quantile_q8 = xr.open_dataarray(file_path)
else:
    calculated_swe_nearest_station_quantile_q8 = xr.full_like(swe.SWE_Post, np.nan)
    for year in np.arange(0,32):
        quantile_fn = f'../data/quantile_ncs/sr_quantiles_{year+1990}_q8.nc'
        sr_quantiles_yr = xr.open_dataarray(quantile_fn)
        calculated_swe_nearest_station_quantile_q8[year] = sr_quantiles_yr.interp(quantile=nearest_station_quantile.SWE_Post[year], method='linear')
    calculated_swe_nearest_station_quantile_q8.to_netcdf(file_path)

### This notebook is missing cumulative precip calculations